In [ ]:
from torch_geometric.datasets import PPI
import torch_geometric.transforms as T

# load the PPI dataset
dataset = PPI(root='data/PPI', split='train', transform=T.Compose([
    T.NormalizeFeatures(),
    T.AddTrainValTestMask(split='train', num_classes=4)
]))

# split the dataset into training, validation, and test sets
train_data = dataset[dataset.train_mask]
val_data = dataset[dataset.val_mask]
test_data = dataset[dataset.test_mask]

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# define the model
class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_size)
        self.conv2 = GCNConv(hidden_size, num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# initialize the model and optimizer
model = GCN(num_features=dataset.num_features, hidden_size=256, num_classes=dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# train the model
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    out = model(train_data.x, train_data.edge_index)
    loss = F.nll_loss(out[train_data.train_mask], train_data.y[train_data.train_mask])
    loss.backward()
    optimizer.step()
    model.eval()
    with torch.no_grad():
        pred = model(val_data.x, val_data.edge_index).argmax(dim=1)
        acc = (pred[val_data.val_mask] == val_data.y[val_data.val_mask]).float().mean()
        print(f"Epoch {epoch + 1:03d}, Val Acc: {acc:.4f}")